# Проект по Python for Data Analysis

Датасет состоит из данных сервиса Яндекс Недвижимость — архив объявлений о продаже квартир в Санкт-Петербурге и соседних населённых пунктах за несколько лет.
По каждой квартире на продажу доступны два вида данных. Первые вписаны пользователем, вторые — получены автоматически на основе картографических данных. Например, расстояние до центра, аэропорта, ближайшего парка и водоёма.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('real_estate_data.csv', encoding='utf-8', sep='\t')

## Этап 1. Предварительная обработка

### Общий обзор данных

In [30]:
pd.set_option('display.max_columns',50)
pd.set_option('display.max_rows', None)
df.head(5)


,total_images,last_price,total_area,first_day_exposition,rooms,ceiling_height,floors_total,living_area,floor,is_apartment,studio,open_plan,kitchen_area,balcony,locality_name,airports_nearest,cityCenters_nearest,parks_around3000,parks_nearest,ponds_around3000,ponds_nearest,days_exposition
0,20,13000000.0,108.0,2019-03-07T00:00:00,3,2.70,16.0,51.0,8,NaN,False,False,25.0,NaN,Санкт-Петербург,18863.0,16028.0,1.0,482.0,2.0,755.0,NaN
1,7,3350000.0,40.4,2018-12-04T00:00:00,1,NaN,11.0,18.6,1,NaN,False,False,11.0,2.0,посёлок Шушары,12817.0,18603.0,0.0,NaN,0.0,NaN,81.0
2,10,5196000.0,56.0,2015-08-20T00:00:00,2,NaN,5.0,34.3,4,NaN,False,False,8.3,0.0,Санкт-Петербург,21741.0,13933.0,1.0,90.0,2.0,574.0,558.0
3,0,64900000.0,159.0,2015-07-24T00:00:00,3,NaN,14.0,NaN,9,NaN,False,False,NaN,0.0,Санкт-Петербург,28098.0,6800.0,2.0,84.0,3.0,234.0,424.0
4,2,10000000.0,100.0,2018-06-19T00:00:00,2,3.03,14.0,32.0,13,NaN,False,False,41.0,NaN,Санкт-Петербург,31856.0,8098.0,2.0,112.0,1.0,48.0,121.0


In [ ]:
num_cols = df.select_dtypes(include=['number']).columns
num_cols


In [ ]:
# df.info()
# df.isnull().sum()

num_cols = df.select_dtypes(include=['number']).columns.tolist()
num_cols = [i for i in num_cols if i not in ('total_images', 'floors_total')]


def show_numeric_plots(data, columns):
    num_cols_len = len(columns)

    _, axs = plt.subplots(num_cols_len, 2, figsize=(12, num_cols_len * 6))

    for i, col in enumerate(columns):
        sns.histplot(data=data[col], ax=axs[i, 0])
        axs[i, 0].set_title(f'{col}')

        sns.boxplot(data=data[col], ax=axs[i, 1])
        axs[i, 1].set_title(f'{col}')

    plt.tight_layout()
    plt.show()


show_numeric_plots(df, num_cols)

In [37]:
df.groupby(['total_area', 'rooms', 'cityCenters_nearest'])['last_price'].mean()
# df.head(5)

total_area  rooms  cityCenters_nearest
12.00       1      964.0                  2.400000e+06
13.00       1      1242.0                 3.800000e+06
                   5483.0                 1.850000e+06
                   15945.0                1.400000e+06
13.20       1      4165.0                 1.686000e+06
14.00       1      11122.0                1.190000e+06
15.00       1      16376.0                1.550000e+06
15.50       0      4589.0                 2.450000e+06
16.00       0      2111.0                 2.100000e+06
17.00       0      29846.0                1.500000e+06
            1      5639.0                 1.900000e+06
17.20       1      13513.0                2.050000e+06
17.60       1      1967.0                 3.400000e+06
17.78       1      12086.0                1.737000e+06
18.00       0      15772.0                1.900000e+06
            1      4948.0                 3.300000e+06
                   9210.0                 2.190000e+06
18.40       1      30687.0

In [ ]:
#Вывод количества пропущенных значений в каждом столбце
null_value_in_column = df.isnull().sum()
for i, (column, null_count) in enumerate(null_value_in_column.sort_values(ascending=False).items(), start=1):
    if null_count > 0:
        percent = round(null_count * 100 / len(df), 1)
        print(f"{i}. {column}: пропусков {null_count} ({percent}%)")

In [ ]:
# График пустых значений
fig, ax = plt.subplots(figsize=(20,12))
sns_heatmap = sns.heatmap(df.isnull(), yticklabels=False, cbar=False, cmap='viridis')

In [ ]:
#проверка на дубликаты
print(f'Количество строк дубликатов: {df.duplicated().sum()}') 

In [ ]:
#сохраним размера df для сравнения после обработки
len_df_start = len(df)

### Детальный обзор каждого параметра

В анализе каждого параметра выполнены следующие шаги:

    1. Обзор всех уникальных значений для визуальной оценки корректности данных данных. 
    2. Подсчет кол-ва пропусков. Где было возможно, сделаны замены пропусков.
    3. Построение гистограммы для поиска аномалий и в некоторых случаях удаление выбросов.
    4. Замена типа данных, где было необходимо.

Результаты обработки по каждому параметру:
1. total_images - удалены строки с редкими значениями (99% персентиль)
2. last_price - удалены строки с редкими значениями (99% персентиль), значения округлены до целого числа.
3. total_area - удалены строки с редкими значениями (99% персентиль)
4. first_day_exposition - удалена временная метка, осталась только дата.
5. rooms - значение 0 заменено на 1, удалены строки с редкими значениями (99% персентиль)
6. ceiling_height - пропущенные значения заменены на среднее значение, удалены строки с редкими значениями (99% персентиль)
7. floors_total - приведен к типу int, удалены все строки с пропусками, удалены строки с редкими значениями (99% персентиль)
8. living_area - пропуски заменены на среднее значение этого параметра с той же общей площадью, удалены строки с редкими значениями (99% персентиль), округение до 2-знаков после запятой
9. floor - без изменений
10. is_apartment - столбец удален, так как пропущено 88% значений, неизвестно на какое значение заменить пропуски.
11. studio - без изменений
12. open_plan - без изменений
13. balcony - заменены пропущенные значения на 0 (нет балкона), приведен к типу int
14. kitchen_area - пропуски заменены на среднюю долю кухни от нежилой зоны по всем строкам
15. airports_nearest - без измнений, в том числе оставлены пропуски
16. cityCenters_nearest - без измнений, в том числе оставлены пропуски
17. days_exposition -  заменены пропущенные значения на 0, приведен к типу int
18. locality_name - заменены дубликаты названий, пропуски оставлены
19. parks_nearest - сделаны замены пропусков на 3100 в тех строках, где известно, что в радиусе 3 км нет парка, остались пропуски
20. ponds_nearest - сделаны замены пропусков на 3100 в тех строках, где известно, что в радиусе 3 км нет водоема, остались пропуски
21. parks_around3000 - без изменений, остались пропуски
22. ponds_around3000 - без изменений, остались пропуски

#### total_images

In [ ]:
# просмотр уникальных значений
df['total_images'].value_counts()

In [ ]:
# количество пустых значений
print(f'Количество пустых значений {df['total_images'].isnull().sum()}')

In [ ]:
#поиск аномалий
df['total_images'].hist(bins=30, log=True)

In [ ]:
#Уберем 1% аномальных значений. Воспользуемся 99% персентилем
threshold = df['total_images'].quantile(0.99)
df = df[df.total_images <= threshold]
# Визуализируем
df['total_images'].hist(bins=30, log=True)

#### last_price

In [ ]:
#Просмотрим уникальные значения
df['last_price'].value_counts()

In [ ]:
#Количество пустых значений
print(f'Количество пустых значений {df['last_price'].isnull().sum()}')

In [ ]:
#Поиск аномалий
df['last_price'].hist(bins=30, log=True)

In [ ]:
#Уберем 1% аномальных значений. Воспользуемся 99% персентилем
threshold = df['last_price'].quantile(0.99)
df = df[df.last_price <= threshold]

# Визуализируем
df['last_price'].hist(bins=30, log=True)

In [ ]:
# Округление до целого
df['last_price'] = df['last_price'].round(0)

#### total_area

In [ ]:
#Уникальные значения
df['total_area'].value_counts()

In [ ]:
#Количество пустых значений
print(f'Количество пустых значений {df['total_area'].isnull().sum()}')

In [ ]:
#Поиск аномалий
df['total_area'].hist(bins=30, log=True)

In [ ]:
#Уберем 1% аномальных значений. Воспользуемся 99% персентилем
threshold = df['total_area'].quantile(0.99)
df = df[df.total_area <= threshold]

# Визуализируем
df['total_area'].hist(bins=30, log=True)

#### first_day_exposition

In [ ]:
#Уникальные значения
df['first_day_exposition'].value_counts()

In [ ]:
#Количество пустых значений
print(f'Количество пустых значений {df['first_day_exposition'].isnull().sum()}')

In [ ]:
#Оставим только дату, так как все временные метки одинаковы
df['first_day_exposition'] = pd.to_datetime(df['first_day_exposition']).dt.normalize()

#### rooms

In [ ]:
# Уникальные значения
df['rooms'].value_counts()

In [ ]:
# Количество пустых значений
print(f'Количество пустых значений {df['rooms'].isnull().sum()}')

Есть строки с числом комнат 0, что не может быть. Проанализируем и сделаем замену:

In [ ]:
#Сгруппируем кол-во комнат по размеру жилой зоны
df.groupby('rooms')['living_area'].mean()

In [ ]:
#Заменим число комнат с 0 на 1, так как они имеют схожее значение по параметру размер жилой площади
df['rooms'] = df['rooms'].replace(0, 1)

In [ ]:
#поиск аномалий
df['rooms'].hist(bins=30, log=True)

In [ ]:
#Уберем 1% аномальных значений. Воспользуемся 99% персентилем
threshold = df['rooms'].quantile(0.99)
df = df[df.rooms <= threshold]

# Визуализируем
df['rooms'].hist(bins=30, log=True)

#### ceiling_height

In [ ]:
#Уникальные значения
df['ceiling_height'].value_counts()

In [ ]:
# В уникальных значениях высоты потолока есть двузначные значения 24.00, вероятно, здесь ошибка в указании десятой части. Сделаем замену
df['ceiling_height'] = df['ceiling_height'].apply(lambda x: x / 10 if x > 10 else x)

In [ ]:
# Количество пустых значений
print(f'Количество пустых значений {df['ceiling_height'].isnull().sum()}')

In [ ]:
print(f'Cреднее = {df['ceiling_height'].mean()}')
print(f'Медиана = {df['ceiling_height'].median()}')

In [ ]:
#Заменим пустые значения средним по всей выборке
df['ceiling_height'] = df['ceiling_height'].fillna(round(df['ceiling_height'].mean(),2))

In [ ]:
#Поиск аномалий
df['ceiling_height'].hist(bins=30, log=True)

In [ ]:
#Уберем 1% аномальных значений. Воспользуемся 99% персентилем
threshold = df['ceiling_height'].quantile(0.99)
df = df[df.ceiling_height <= threshold]

# Визуализируем
df['ceiling_height'].hist(bins=30)

#### floors_total

In [ ]:
# просмотр уникальных значений
df['floors_total'].value_counts()

In [ ]:
# Количество пустых значений
print(f'Количество пустых значений {df['floors_total'].isnull().sum()}')

In [ ]:
#Удалим строки с пустыми значенями, так как их меньше 0,5%
df = df.dropna(subset=['floors_total'])

In [ ]:
#Приведем значения к типу int
df['floors_total'] = df.floors_total.astype(int)

In [ ]:
#Поиск аномалий
df['floors_total'].hist(bins=30, log=True)

In [ ]:
#Уберем 1% аномальных значений. Воспользуемся 99% персентилем
threshold = df['floors_total'].quantile(0.99)
df = df[df.floors_total <= threshold]

# Визуализируем
df['floors_total'].hist(bins=30, log=True)

#### living_area 

In [ ]:
# Просмотр уникальных значений
df['living_area'].value_counts()

In [ ]:
# Проверка на пропущенные значения
print(f'Количество пропущенных значений: {df['living_area'].isna().sum()}')

In [ ]:
# Найдем среднее значение жилой зоны для округленного значения всей площади
df.groupby(df['total_area'].round())['living_area'].mean()

In [ ]:
# Замена пропусков средним значением для тех же total_area
df['living_area'] = df.groupby(df['living_area'].round())['living_area'].transform(lambda group: group.fillna(group.mean()))

Остались пропуски в нескольких living_area. В строках с total_area 15.0 сделаем замену на среднее значение в living_area с total_area 14.0 

In [ ]:
df.loc[df['total_area'].round() == 15, 'living_area'] = 11

In [ ]:
#Поиск выбросов
df['living_area'].hist(bins=30, log=True)

In [ ]:
#Уберем 1% аномальных значений. Воспользуемся 99% персентилем
threshold = df['living_area'].quantile(0.99)
df = df[df['living_area'] <= threshold]

# Визуализируем
df['living_area'].hist(bins=30, log=True)

In [ ]:
#округлим до двух знаков после запятой
df['living_area'] = df['living_area'].round(2)

#### floor

In [ ]:
# Просмотр уникальных значений
df['floor'].value_counts()

In [ ]:
# проверка на пропущенные значения
print(f'Количество пропущенных значений: {df.floor.isna().sum()}')

#### is_apartment

In [ ]:
#Уникальные значения
df['is_apartment'].value_counts() 

In [ ]:
# Проверка на пропущенные значения
print(f'Количество пропущенных значений: {df['is_apartment'].isna().sum()}')

Невозможно определить значение. Так как остается большое количество пропущенных строк, то удалим этот столбец.

In [ ]:
df.drop(columns=['is_apartment'], inplace=True)

#### studio

In [ ]:
# Просмотр уникальных значений
df['studio'].value_counts()

In [ ]:
# Проверка на пропущенные значения
print(f'Количество пропущенных значений: {df['studio'].isna().sum()}')

#### open_plan

In [ ]:
# Просмотр уникальных значений
df['open_plan'].value_counts()

In [ ]:
# проверка на пропущенные значения
print(f'Количество пропущенных значений: {df['open_plan'].isna().sum()}')

#### balcony

In [ ]:
# Просмотр уникальных значений
df['balcony'].value_counts()

In [ ]:
#Проверяем кол-во пропусков
print(f'Кол-во пропусков в balcony: {df['balcony'].isna().sum()}')

In [ ]:
# Заменим отсутствующие значения в столбце balcony на 0
df['balcony'] = df['balcony'].fillna(0)

In [ ]:
# Поменяем тип данных на int
df['balcony'] = df['balcony'].astype(int)

#### kitchen_area

In [ ]:
# Просмотр значений
df['kitchen_area'].value_counts()

In [ ]:
# Проверка на пропущенные значения
print(f'Количество пропущенных значений: {df['kitchen_area'].isna().sum()}')

Определим значение для замены пропусков: рассчитаем для каждой строки разницу между total_area и living_area, а затем рассчитаем какой процент от нежилой зоны в среднем занимает кухня

In [ ]:
# доля, которую в среднем занимает кухня от нежилой территории
median_kitchen_percentage = round((df['kitchen_area'] / (df['total_area'] - df['living_area'])).median(), 1)
print(f'Доля кухни от нежилой зоны в среднем равна {median_kitchen_percentage}')

In [ ]:
#для каждой строки, где известы total_area и living_area, получим значение для kitchen_area
df.loc[df['kitchen_area'].isna(), 'kitchen_area'] = df['total_area'] - df['living_area'] * median_kitchen_percentage

#### airports_nearest

In [ ]:
# Просмотр уникальных значений
df['airports_nearest'].value_counts()

In [ ]:
# Проверка на пропущенные значения
print(f'Количество пропущенных значений: {df['airports_nearest'].isna().sum()}')

In [ ]:
#Оставим незаполненными, так как нет возможности заполнить

#### cityCenters_nearest

In [ ]:
# Просмотр уникальных значений
df['cityCenters_nearest'].value_counts()

In [ ]:
# Проверка на пропущенные значения
print(f'Количество пропущенных значений: {df['cityCenters_nearest'].isna().sum()}')

In [ ]:
#Оставим незаполненными, так как нет возможности заполнить

#### days_exposition

In [ ]:
# Просмотр уникальных значений
df['days_exposition'].value_counts()

In [ ]:
# Проверка на пропущенные значения
print(f'Количество пропущенных значений: {df['days_exposition'].isna().sum()}')

In [ ]:
#Замена пропусков
df['days_exposition'] = df['days_exposition'].fillna(0)

In [ ]:
#Изменение типа данных
df['days_exposition'] = df['days_exposition'].astype('int')

In [ ]:
#Поиск выбросов
df['days_exposition'].hist(bins=30, log=True)

#### locality_name

In [ ]:
# Просмотр уникальных значений
df['locality_name'].unique()

In [ ]:
print(f'Количество уникальных значений до замены: {len(df['locality_name'].unique())}')

In [ ]:
#замена названий
names_variant = {'посёлок': 'поселок',
                 'посёлок городского типа': 'поселок',
                 'поселок городского типа': 'поселок',
                 'городской поселок': 'поселок',
                 'городской посёлок': 'поселок',
                 'поселок при железнодорожной станции': 'поселок',
                 'посёлок при железнодорожной станции': 'поселок',
                 'поселок станции': 'поселок',
                 'посёлок станции': 'поселок'}
df['locality_name'] = df['locality_name'].replace(names_variant, regex=True)

In [ ]:
print(f'Количество уникальных значений после замены: {len(df['locality_name'].unique())}')

In [ ]:
# Проверка на пропущенные значения
print(f'Количество пропущенных значений: {df['locality_name'].isna().sum()}')

#### parks_nearest и parks_around3000

In [ ]:
# Просмотр уникальных значений
df['parks_nearest'].value_counts()

In [ ]:
df['parks_around3000'].value_counts()

Проверим зависимость между столбцами "parks_around3000" - число парков в радиусе 3 км и "parks_nearest" - расстояние до ближайшего парка (м).

In [ ]:
filtered_df = df[(df['parks_around3000'].notna()) & (df['parks_nearest'].notna())]
x = filtered_df['parks_around3000']
y = filtered_df['parks_nearest']

sns.scatterplot(data=filtered_df, x=x, y=y)

plt.xticks(range(int(filtered_df['parks_around3000'].min()), int(filtered_df['parks_around3000'].max()) + 1))
plt.show()

In [ ]:
filtered_df = df[(df['parks_around3000'] > 0)]
print(f'Если значение в parks_around3000 > 0, то пропущенных в parks_nearest {len(filtered_df[(pd.isna(filtered_df['parks_nearest']))])}, максимальное значение равно {filtered_df['parks_nearest'].max()}') 

In [ ]:
filtered_df = df[(df['parks_around3000'] == 0)]
print(f'Если значение в parks_around3000 == 0, то пропущенных в parks_nearest {len(filtered_df[(pd.isna(filtered_df['parks_nearest']))])}, максимальное значение равно {filtered_df['parks_nearest'].max()}') 

In [ ]:
filtered_df = df[(pd.isna(df['parks_around3000']))]
print(f'Если значение в parks_around3000 пропущено, то пропущенных в parks_nearest: {filtered_df['parks_nearest'].isna().sum()}, максимальное значение равно {filtered_df['parks_nearest'].max()}')

In [ ]:
filtered_df = df[(pd.notna(df['parks_nearest']))]
print(f'Если значение в parks_nearest заполнено, то пропущенных в parks_around3000: {filtered_df['parks_around3000'].isna().sum()}, максимальное значение равно {filtered_df['parks_around3000'].max()}')

Исходя из результатов выше, можно разбить данные с пропусками в parks_nearest на две группы:

1 группа: известно, что в радиусе 3 км нет парка

2 группа: неизвестно, есть ли в радиусе 3 км парк, так как значение в parks_around3000 пропущено

Сделаем замену для первой группы в поле parks_nearest и укажем расстояние больше 3 км, например, 3100.
Для второй группы оставим пропуски.

In [ ]:
#Известно, что в радиусе 3 км нет парка. Делаем замену:
filtered_df = df[(df['parks_around3000'] == 0)]
df.loc[(df['parks_nearest'].isna()) & (df['parks_around3000'] == 0), 'parks_nearest'] = 3100

In [ ]:
#проверяем кол-во пропусков после замен
print(f'Кол-во пропусков в parks_around3000: {df['parks_around3000'].isna().sum()}')
print(f'Кол-во пропусков в parks_nearest: {df['parks_nearest'].isna().sum()}')

#### ponds_nearest и ponds_around3000

In [ ]:
# Просмотр уникальных значений
df['ponds_nearest'].value_counts()

In [ ]:
df['ponds_around3000'].value_counts()

Все шаги выполним аналогично тому, что сделали для двух параметров выше:
Проверим зависимость между столбцами "ponds_around3000" - число водоемов в радиусе 3 км и "ponds_nearest" - расстояние до ближайшего водоема (м).

In [ ]:
filtered_df = df[(df['ponds_around3000'] > 0)]
print(f'Если значение в ponds_around3000 > 0, то пропущенных в ponds_nearest {len(filtered_df[(pd.isna(filtered_df['ponds_nearest']))])}, максимальное значение равно {filtered_df['ponds_nearest'].max()}') 

In [ ]:
filtered_df = df[(df['ponds_around3000'] == 0)]
print(f'Если значение в ponds_around3000 == 0, то пропущенных в ponds_nearest {len(filtered_df[(pd.isna(filtered_df['ponds_nearest']))])}, максимальное значение равно {filtered_df['ponds_nearest'].max()}') 

In [ ]:
filtered_df = df[(pd.isna(df['ponds_around3000']))]
print(f'Если значение в ponds_around3000 пропущено, то пропущенных в ponds_nearest: {filtered_df['ponds_nearest'].isna().sum()}, максимальное значение равно {filtered_df['ponds_nearest'].max()}')

In [ ]:
filtered_df = df[(pd.notna(df['ponds_nearest']))]
print(f'Если значение в parks_nearest заполнено, то пропущенных в ponds_around3000: {filtered_df['ponds_around3000'].isna().sum()}, максимальное значение равно {filtered_df['ponds_around3000'].max()}')

Cделаем замены по аналогии со столбцами "parks_around3000" и "parks_nearest"

In [ ]:
#1 Так как в ponds_nearest всегда NaN, когда ponds_around3000 = 0, заменим в этом случае пропуски фиксированным значением 3100
df.loc[(df['ponds_nearest'].isna()) & (df['ponds_around3000'] == 0), 'ponds_nearest'] = 3100

In [ ]:
#проверяем кол-во пропусков после замен
print(f'Кол-во пропусков в ponds_around3000: {df['ponds_around3000'].isna().sum()}')
print(f'Кол-во пропусков в ponds_nearest: {df['ponds_nearest'].isna().sum()}')

### Результаты обработки

In [ ]:
print(f'Количество строк до обработки = {len_df_start}, количество строк после обработки = {len(df)}')

In [ ]:
#Карта пропусков
fig, ax = plt.subplots(figsize=(20,12))
sns_heatmap = sns.heatmap(df.isnull(), yticklabels=False, cbar=False, cmap='viridis')

In [ ]:
df.info()

## Этап 2. Добавление новых признаков

Добавим новые временные признаки (год, месяц, день недели публикации объявления), признак цена за квадратный метр, тип этажа квартиры (первый, последний, первый и последний, другой)

In [ ]:
df['year'] = df['first_day_exposition'].dt.year
df['month'] = df['first_day_exposition'].dt.month
df['weekday'] = df['first_day_exposition'].dt.weekday
df['price_per_m2'] = (df['last_price'] / df['total_area']).round(2)
df['floor_type'] = df.apply(lambda row: 'Первый и последний' if row['floor'] == 1 and row['floors_total'] == 1 else ('Первый' if row['floor'] == 1 else ('Последний' if row['floor'] == row['floors_total'] else 'Другой')),axis=1)

In [ ]:
df.head()

In [ ]:
#сохраним обработтанве данные в новый датасет
df.to_csv('real_estate_processed.csv', index=False)